# Защита персональных данных клиентов

## Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

### Инструкция по выполнению проекта

1. Загрузите и изучите данные.  
2. Ответьте на вопрос и обоснуйте решение.  
3. Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)   
    a. Изменится. Приведите примеры матриц.  
    b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.  
4. Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.  
5. Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.  

### Описание данных
Набор данных находится в файле /datasets/insurance.csv.

    Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
    Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Libs

In [31]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### 1. Загрузите и изучите данные.

In [4]:
data = pd.read_csv('insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [7]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [8]:
data['Пол'].value_counts()

0    2505
1    2495
Name: Пол, dtype: int64

In [9]:
data['Члены семьи'].value_counts()

1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: Члены семьи, dtype: int64

#### Проверим дубли

In [13]:
data.duplicated().sum()

153

In [16]:
data = data.drop_duplicates()

### Проверим на пустые строки

In [17]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [21]:
data['Возраст'] = data['Возраст'].astype(int)
data['Зарплата'] = data['Зарплата'].astype(int)

<ipython-input-21-becc5e8ac170>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Возраст'] = data['Возраст'].astype(int)
<ipython-input-21-becc5e8ac170>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Зарплата'] = data['Зарплата'].astype(int)


В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения (нахлждения весов):

$$
w = (X^T X)^{-1} X^T y
$$

Формула расчета весов для линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

Новая матрица = $X'$ как произведение старой $X$ на матрицу $P$ (обратимую):

$$
X' = X * P
$$

Подставим новое значение $X'$ в формулу $w'$:

$$
w' = ((X P)^T X P)^{-1} (X P)^T y
$$

Если $ (AB)^T = A^T *B^T  $, тогда:

$$
w' = (P^T X^T X P)^{-1} P^T X^T y
$$

Если $ (AB)^-1 = B^-1 *A^-1  $, тогда:

$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

$P$ по условию обратимая => произведение $(P^T)^{-1} P^T$ равно $E$:

$$
w' = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$



$$
w' = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$


Предсказания модели расчитываются по формуле:

$$
a = Xw
$$

Подставим в эту формулу значения $X'$ и $w'$ для того, чтобы на их основе вычислить предсказания модели $a'$:

$$
a' = X'w' = XPP^{-1} w
$$

Т.к $P$ по условию обратимая, то произведение $PP^{-1}$ = $E$:

$$
a' = XPP^{-1} w = X E w = X w = a
$$

$$
w' = P^{-1} w
$$

$$
a' = Xw
$$
$$
a' = a
$$

### Вывод:
При умножении признаков на обратимую матрицу, предсказания модели не поменяются.
b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.


### 3. Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

#### Ответ. Сгенерируем из случайных чисел матрицу размерностью 4х4 (т.к в датасете имеет 4 анализуемых признака), которую скалярно умножим на матрицу признаков. Затем рассчитаем MSE и r2_score и сравним их.

### 4. Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2. 

In [46]:
# Разделим выборку на тренировочную и тестовую
train, test = train_test_split(data, test_size = .3, random_state=2021)

In [26]:
# Выделим признаки
feature_train = train.drop('Страховые выплаты', axis=1)
target_train = train['Страховые выплаты']

In [47]:
feature_test = test.drop('Страховые выплаты', axis=1)
target_test = test['Страховые выплаты']

In [54]:
# Обучим модель линейной регресии и рассчитаем метрику R2 по исходному датасету
model = LinearRegression()
model.fit(feature_train,target_train)
predictions = model.predict(feature_test)
mse = mean_squared_error(target_test, predictions)
print('MSE Score:', mse)
print("R2 score :", r2_score(target_test, predictions))

MSE Score: 0.14300196233921397
R2 score : 0.4529635404087079


#### Сгенерируем из случайных чисел матрицу размерностью 4х4, которую скалярно умножим на матрицу признаков.

In [62]:
# Создадим случайную матрицу размером 4х4
random_matrix = np.random.normal(1, 5000, size = (4,4))

#Выведем на печать инвертированную матрицу random_matrix
print(np.linalg.inv(random_matrix))
# Инвертация выполнилась

# Найдем скалярное произведение матрицы из случайных чисел и признаков тренировочной выборки
feature_train_transform = feature_train.dot(random_matrix)

# Найдем скалярное произведение матрицы из случайных чисел и признаков тестовой выборки
feature_test_transform = feature_test.dot(random_matrix)

# Обучим модель на преобразованных данных
model.fit(feature_train_transform, target_train)


predictions2 = model.predict(feature_test_transform)

mse2 = mean_squared_error(target_test, predictions2)
print('MSE Score:', mse)
print("R2 score transform:", r2_score(target_test, predictions2))


[[-8.91267803e-05  1.41705490e-04  3.37177444e-05 -1.35153773e-04]
 [ 2.06265423e-04  5.73194068e-04  5.19091024e-05  8.72277120e-05]
 [ 3.57855466e-05 -1.31523039e-04  4.78877637e-05  1.55792613e-05]
 [-1.77864959e-05 -1.20084275e-05  2.34728693e-05  6.13677573e-05]]
MSE Score: 0.14300196233921397
R2 score transform: 0.45296354040915987


### Вывод: После проверки на тестовой выборки можно увидеть, что трансформированная и исходная матрицы имеют одинаковые показатели R2

In [38]:
#Так как по условию задачи эта матрица при умножении на матрицу признаков должна давать матрицу такого же размера
#как исходная, то $P$ должна быть квадратной с размером, равным количеству признаков, в нашем случае их 4.
random_matrix = np.random.randint(100,size = (4, 4))
random_matrix

array([[32, 70, 25, 38],
       [20, 43, 40, 88],
       [24, 49, 26, 51],
       [79, 16, 70, 82]])

In [39]:
#Проверим матрицу на обратимость 
matrix_inverted = np.linalg.inv(random_matrix)
matrix_inverted

array([[-0.1506586 , -0.13237586,  0.32902935,  0.00723909],
       [ 0.04154223,  0.01740861, -0.05248787, -0.00528879],
       [ 0.30843226,  0.22849027, -0.64547682,  0.013314  ],
       [-0.12625494, -0.07091664,  0.24426664, -0.00511277]])

In [43]:
from sklearn.preprocessing import StandardScaler

In [44]:
#Обозначим признаки и целевой признак
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']
 
#Проведем масштабирование с помощью StandardScaler()
scaler = StandardScaler()
scaler.fit(features)

#Сохраним масштабированные признаки в features_scaled
features_scaled = scaler.transform(features)

In [45]:
#Расчитаем веса $w$ для исходных признаков (масштабированных) по стандартной формуле $w = (X^T X)^{-1} X^T y$
w = np.linalg.inv(features_scaled.T.dot(features_scaled)).dot(features_scaled.T).dot(target)

#Расчитаем вектор предсказаний по исходным признакам по формуле 𝑎=𝑋𝑤
a = features_scaled @ w

#Проведем преобразование признаков:
features_p = features_scaled @ random_matrix

#Расчитаем веса  𝑤1  для преобразованных признаков по стандартной формуле  𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦
w1 = np.linalg.inv(features.T.dot(features_scaled)).dot(features_scaled.T).dot(target)

#Расчитаем вектор предсказаний по преобразованным признакам по формуле  𝑎=𝑋𝑤
a1 = features_p @ w1

#Получим вектор разниц между предсказаниями по исходным признакам  𝑎  и преобразованным  𝑎1 :
diff = a - a1

#Просуммируем все элементы вектора diff, чтобы убедиться, что расхождения минимальны.
diff.sum()

1.9397816686250735e-12